In [ ]:
!pip install --quiet transformers
!pip install --quiet pytorch-lightning==1.2.7
!pip install sentencepiece

     |████████████████████████████████| 3.4 MB 8.1 MB/s 
     |████████████████████████████████| 596 kB 40.8 MB/s 
     |████████████████████████████████| 3.3 MB 34.2 MB/s 
     |████████████████████████████████| 895 kB 46.1 MB/s 
     |████████████████████████████████| 61 kB 452 kB/s 
     |████████████████████████████████| 830 kB 5.4 MB/s 
     |████████████████████████████████| 829 kB 34.2 MB/s 
     |████████████████████████████████| 332 kB 44.3 MB/s 
     |████████████████████████████████| 132 kB 45.7 MB/s 
     |████████████████████████████████| 1.1 MB 35.2 MB/s 
     |████████████████████████████████| 160 kB 42.8 MB/s 
     |████████████████████████████████| 271 kB 38.1 MB/s 
     |████████████████████████████████| 192 kB 48.1 MB/s 
     |████████████████████████████████| 1.2 MB 5.3 MB/s 


In [ ]:
import pandas as pd
import csv

In [ ]:
import torch
from transformers import MT5ForConditionalGeneration, T5Tokenizer
import pandas as pd
import csv
from collections import Counter
from tqdm.auto import tqdm, trange


tokenizer = T5Tokenizer.from_pretrained("google/mt5-base")
model = MT5ForConditionalGeneration.from_pretrained('google/mt5-base')

Downloading:   0%|          | 0.00/4.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/376 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/702 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.17G [00:00<?, ?B/s]

In [ ]:
def msize(m):
    return sum(p.numel() for p in m.parameters())
print(msize(model.shared) / msize(model))   # 0.3298
print(msize(model.lm_head) / msize(model))  # 0.3298

0.32981729710484153
0.32981729710484153


In [ ]:
train_df1=pd.read_csv('/content/drive/MyDrive/models/Arabic_data/arabicTrain.csv')
train_df2=pd.read_csv('/content/drive/MyDrive/models/ar_dial.zip (Unzipped Files)/out.csv')

test_df=pd.read_csv('/content/drive/MyDrive/models/Arabic_data/arabicTest.csv')

val_df=pd.read_csv('/content/drive/MyDrive/models/Arabic_data/arabicVal.csv')


In [ ]:
train_df1.drop(index=8197,inplace=True)

In [ ]:
train_df1.reset_index(drop=True, inplace=True)
train_df2.reset_index(drop=True, inplace=True)

In [ ]:
train_df = pd.concat([train_df1, train_df2], axis=1)

In [ ]:
train_df

,Unnamed: 0,summary,dialogue,ar_summary,summary,dialogue,len,ar_dialogue
0,0,Amanda baked cookies and will bring Jerry some...,Amanda: I baked cookies. Do you want some?\r\...,أماندا خبزت كعكات وستحضر جيري بعضا منها غدا,Amanda baked cookies and will bring Jerry some...,|Amanda: I baked cookies. Do you want some? |...,94,| أماندا: لقد خبزت ملفات تعريف الارتباط. هل تر...
1,1,Olivia and Olivier are voting for liberals in ...,Olivia: Who are you voting for in this electio...,أوليفيا وأوليفييه يصوتون لليبراليين في هذه الا...,Olivia and Olivier are voting for liberals in ...,|Olivia: Who are you voting for in this electi...,111,| أوليفيا: لمن تصوت في هذه الانتخابات؟ | أوليف...
2,2,Kim may try the pomodoro technique recommended...,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa...",قد يجرب كيم تقنية بومودورو التي أوصى بها تيم ل...,Kim may try the pomodoro technique recommended...,"|Tim: Hi, what is up? |Kim: Bad mood tbh, I wa...",528,| تيم: مرحبًا ، ما الأمر؟ | كيم: الحالة المزاج...
3,3,Edward thinks he is in love with Bella. Rachel...,"Edward: Rachel, I think I'm in ove with Bella....",يعتقد إدوارد أنه يحب بيلا. تريد راشيل أن يفتح ...,Edward thinks he is in love with Bella. Rachel...,"|Edward: Rachel, I think I am in ove with Bell...",155,| إدوارد: راشيل ، أعتقد أنني في حالة حب مع بيل...
4,4,"Sam is confused, because he overheard Rick com...",Sam: hey overheard rick say something\r\nSam:...,"""سام"" مرتبك ، لأنه سمع ريك يشكو منه كزميل في ا...","Sam is confused, because he overheard Rick com...",|Sam: hey overheard rick say something |Sam: ...,909,| سام: مرحبًا سمع ريك يقول شيئًا | سام: لا أعر...
...,...,...,...,...,...,...,...,...
14725,14727,Romeo is trying to get Greta to add him to her...,Romeo: You are on my ‘People you may know’ lis...,يحاول روميو إقناع غريتا بإضافته إلى قائمة أصدق...,Romeo is trying to get Greta to add him to her...,|Romeo: You are on my ‘People you may know’ li...,297,"| روميو: أنت مدرج في قائمة ""أشخاص قد تعرفهم"".\..."
14726,14728,Theresa is at work. She gets free food and fre...,Theresa: <file_photo>\r\nTheresa: <file_photo>...,تيريزا في العمل. تحصل على طعام مجاني ودروس يوج...,Theresa is at work. She gets free food and fre...,|Theresa: <file_photo> |Theresa: <file_photo> ...,1047,| تيريزا: <file_photo> | تيريزا: <file_photo> ...
14727,14729,Japan is going to hunt whales again. Island an...,John: Every day some bad news. Japan will hunt...,ستقوم اليابان بمطاردة الحيتان مرة أخرى. لم تتو...,Japan is going to hunt whales again. Island an...,|John: Every day some bad news. Japan will hun...,842,| جون: كل يوم بعض الأخبار السيئة. اليابان ستصط...
14728,14730,Celia couldn't make it to the afternoon with t...,Jennifer: Dear Celia! How are you doing?\r\nJe...,لم تستطع سيليا الوصول إلى فترة ما بعد الظهر مع...,Celia couldn't make it to the afternoon with t...,|Jennifer: Dear Celia! How are you doing? |Jen...,1540,| جينيفر: عزيزتي سيليا! كيف هي احوالك؟ | جينيف...


In [ ]:
train_df=train_df[['ar_summary','ar_dialogue']]
test_df=test_df[['ar_summary','ar_dialogue']]
val_df=val_df[['ar_summary','ar_dialogue']]


In [ ]:
val_df

,ar_summary,ar_dialogue
0,"ستذهب ""أ"" إلى مأوى الحيوانات غدًا لتحصل على جر...",ج: مرحبًا توم ، هل أنت مشغول بعد ظهر الغد؟\r\n...
1,تحب إيما وروب التقويم المجيء. يناسب Lauren عنا...,إيما: لقد وقعت للتو في حب تقويم المجيء هذا! مد...
2,ماديسون حامل لكنها لا تريد التحدث عن ذلك. تزوج...,جاكي: ماديسون حامل\r\nجاكي: لكنها لا تريد الحد...
3,وجدت مارلا زوجًا من الملاكمين تحت سريرها.,مارلا: <file_photo>\r\nمارلا: انظر ماذا وجدته ...
4,يريد روبرت من فريد أن يرسل له عنوان متجر الموس...,روبرت: أعطني عنوان متجر الموسيقى هذا الذي ذكرت...
...,...,...
813,موعد تخرج كارلا في الرابع من يونيو. سيحاول ديي...,كارلا: لقد حصلت عليه ...\r\nدييغو: ماذا؟\r\nكا...
814,ستذهب بيف في رحلة مدرسية مع ابنها. جيتا ذاهبة ...,غيتا: مرحبًا ، هذه أمي غيتا بيتي ، أردت أن أسأ...
815,خدع جريج جوليا. يعتذر لها. روبرت يخبر جوليا أل...,جوليا: لقد أرسل لي جريج رسالة نصية\r\nروبرت: آ...
816,كسرت أظافرها الزواج وأقيمت حفلة غدا. ستتصل تين...,الزواج: كسرت ظفر ؛ (\r\nتينا: أوه ، لا!\r\nالز...


In [ ]:
train_df.dropna(inplace=True)
test_df.dropna(inplace=True)
val_df.dropna(inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
train_df.to_csv("/content/drive/MyDrive/models/Arabic_data/train.csv")

In [ ]:
print(tokenizer)

PreTrainedTokenizer(name_or_path='google/mt5-base', vocab_size=250100, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'})


In [ ]:
cnt_ar = Counter()

for text in tqdm(train_df.ar_dialogue):
    cnt_ar.update(tokenizer.encode(text))

for text in tqdm(train_df.ar_summary):
    cnt_ar.update(tokenizer.encode(text))

print(len(cnt_ar), len(cnt_ar)/tokenizer.vocab_size)

  0%|          | 0/14730 [00:00<?, ?it/s]

  0%|          | 0/14730 [00:00<?, ?it/s]

13387 0.0535265893642543


In [ ]:
for top in 12_000, 15_000:
    print(top, sum(v for k, v in cnt_ar.most_common(top)) / sum(cnt_ar.values()))

12000 0.9996191331888767
15000 1.0


In [ ]:
new_tokens = set(range(1000))

for i, (k, v) in enumerate(cnt_ar.most_common(12_000)):
    if len(new_tokens) == 29_900:
        print(i, 'ar tokens are included')
        break
    if k not in new_tokens:
        new_tokens.add(k)
for t in range(tokenizer.vocab_size - 100, tokenizer.vocab_size):
    new_tokens.add(t)
print(len(new_tokens))
kept_ids = sorted(new_tokens)

12642


In [ ]:
new_size = len(kept_ids)
new_emb = torch.nn.Embedding(new_size, model.shared.embedding_dim)
new_head = torch.nn.Linear(in_features=model.lm_head.in_features, out_features=new_size, bias=False)
for new_id, old_id in enumerate(kept_ids):
    new_emb.weight.data[new_id] = model.shared.weight.data[old_id]
    new_head.weight.data[new_id] = model.lm_head.weight.data[old_id]
model.shared.weight = new_emb.weight
model.lm_head.weight = new_head.weight
model.config.__dict__['vocab_size'] = new_size
model.config.__dict__['_name_or_path'] = 'cointegrated/art5-base'

In [ ]:
!pip install sentencepiece
! wget https://raw.githubusercontent.com/google/sentencepiece/master/src/sentencepiece_model.proto
! protoc --python_out=. sentencepiece_model.proto
import sentencepiece_model_pb2 as spmp


--2022-01-02 06:40:53--  https://raw.githubusercontent.com/google/sentencepiece/master/src/sentencepiece_model.proto
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12872 (13K) [text/plain]
Saving to: ‘sentencepiece_model.proto’

sentencepiece_model 100%[===================>]  12.57K  --.-KB/s    in 0s      

2022-01-02 06:40:54 (83.8 MB/s) - ‘sentencepiece_model.proto’ saved [12872/12872]



In [ ]:

import sentencepiece_model_pb2 as spmp
smp = tokenizer.sp_model.serialized_model_proto()
m = spmp.ModelProto()
m.ParseFromString(smp)
print('the loaded model has pieces:', len(m.pieces))
new_pieces = [m.pieces[idx] for idx in kept_ids]
print('the new pieces:', len(new_pieces))
# replace the content of the first 30K pieces
for i, p in enumerate(new_pieces):
    m.pieces[i].piece = p.piece
    m.pieces[i].score = p.score
    m.pieces[i].type = p.type
# drop the remaining pieces
n = len(new_pieces)
for i in trange(len(m.pieces) - n):
    m.pieces.pop(len(m.pieces) - 1)
print(len(m.pieces))
with open('new_sp.model', 'wb') as f:
    f.write(m.SerializeToString())
new_tokenizer = T5Tokenizer('new_sp.model', extra_ids=0)

the loaded model has pieces: 250100
the new pieces: 12642


  0%|          | 0/237458 [00:00<?, ?it/s]

12642


In [ ]:
new_tokenizer.save_pretrained('/content/drive/MyDrive/art5-base')
model.save_pretrained('/content/drive/MyDrive/art5-base')